In [1]:
import pandas as pd
weather = pd.read_csv('nyc_weather_2018.csv')
weather.head()

,date,datatype,station,attributes,value
0,2018-01-01T00:00:00,PRCP,GHCND:US1CTFR0039,",,N,",0.0
1,2018-01-01T00:00:00,PRCP,GHCND:US1NJBG0015,",,N,",0.0
2,2018-01-01T00:00:00,SNOW,GHCND:US1NJBG0015,",,N,",0.0
3,2018-01-01T00:00:00,PRCP,GHCND:US1NJBG0017,",,N,",0.0
4,2018-01-01T00:00:00,SNOW,GHCND:US1NJBG0017,",,N,",0.0


In [2]:
snow_data = weather.query(
    'datatype == "SNOW" and value > 0 '
    'and station.str.contains("US1NY") '
)

snow_data.head()

,date,datatype,station,attributes,value
114,2018-01-01T00:00:00,SNOW,GHCND:US1NYWC0019,",,N,",25.0
789,2018-01-04T00:00:00,SNOW,GHCND:US1NYNS0007,",,N,",41.0
794,2018-01-04T00:00:00,SNOW,GHCND:US1NYNS0018,",,N,",10.0
798,2018-01-04T00:00:00,SNOW,GHCND:US1NYNS0024,",,N,",89.0
800,2018-01-04T00:00:00,SNOW,GHCND:US1NYNS0030,",,N,",102.0


In [3]:
weather[
    (weather.datatype == 'SNOW') & (weather.value > 0) & weather.station.str.contains('US1NY')
].equals(snow_data)

True

In [4]:
station_info = pd.read_csv('weather_stations.csv')
station_info.head()

,id,name,latitude,longitude,elevation
0,GHCND:US1CTFR0022,"STAMFORD 2.6 SSW, CT US",41.064100,-73.577000,36.6
1,GHCND:US1CTFR0039,"STAMFORD 4.2 S, CT US",41.037788,-73.568176,6.4
2,GHCND:US1NJBG0001,"BERGENFIELD 0.3 SW, NJ US",40.921298,-74.001983,20.1
3,GHCND:US1NJBG0002,"SADDLE BROOK TWP 0.6 E, NJ US",40.902694,-74.083358,16.8
4,GHCND:US1NJBG0003,"TENAFLY 1.3 W, NJ US",40.914670,-73.977500,21.6


In [5]:
station_info.id.describe()

count                   279
unique                  279
top       GHCND:US1CTFR0022
freq                      1
Name: id, dtype: object

In [6]:
weather.station.describe()

count                 78780
unique                  110
top       GHCND:USW00094789
freq                   4270
Name: station, dtype: object

In [7]:
station_info.shape[0], weather.shape[0]

(279, 78780)

In [8]:
def get_row_count(*dfs):
    return [df.shape[0] for df in dfs]
get_row_count(station_info, weather)

[279, 78780]

In [9]:
inner_join = weather.merge(station_info, left_on='station', right_on='id')
inner_join.sample(5, random_state=42)

,date,datatype,station,attributes,value,id,name,latitude,longitude,elevation
70525,2018-12-26T00:00:00,WDF5,GHCND:USW00094789,",,W,",280.0,GHCND:USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.63915,-73.764010,3.4
44728,2018-11-11T00:00:00,TMIN,GHCND:USW00014732,",,W,2400",2.2,GHCND:USW00014732,"LAGUARDIA AIRPORT, NY US",40.77944,-73.880350,3.4
57683,2018-05-21T00:00:00,TMAX,GHCND:USW00094728,",,W,2400",26.1,GHCND:USW00094728,"NY CITY CENTRAL PARK, NY US",40.77898,-73.969250,42.7
75689,2018-08-09T00:00:00,PRCP,GHCND:US1NJBG0010,",,N,",1.3,GHCND:US1NJBG0010,"RIVER VALE TWP 1.5 S, NJ US",40.99145,-74.012348,9.4
41042,2018-12-25T00:00:00,SNWD,GHCND:USC00308577,",,7,0800",0.0,GHCND:USC00308577,"SYOSSET, NY US",40.82160,-73.490800,80.8


In [10]:
weather.merge(station_info.rename(dict(id='station'), axis=1), on='station').sample(5, random_state=0)

,date,datatype,station,attributes,value,name,latitude,longitude,elevation
10739,2018-08-07T00:00:00,SNOW,GHCND:US1NJMN0069,",,N,",0.0,"LONG BRANCH 1.7 SSW, NJ US",40.275368,-74.006027,9.4
45188,2018-12-21T00:00:00,TMAX,GHCND:USW00014732,",,W,2400",16.7,"LAGUARDIA AIRPORT, NY US",40.779440,-73.880350,3.4
59823,2018-01-15T00:00:00,WDF5,GHCND:USW00094741,",,W,",40.0,"TETERBORO AIRPORT, NJ US",40.850000,-74.061390,2.7
10852,2018-10-31T00:00:00,PRCP,GHCND:US1NJMN0069,"T,,N,",0.0,"LONG BRANCH 1.7 SSW, NJ US",40.275368,-74.006027,9.4
46755,2018-05-05T00:00:00,SNOW,GHCND:USW00014734,",,W,",0.0,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",40.682500,-74.169400,2.1


In [11]:
left_join = station_info.merge(weather, left_on='id', right_on='station', how='left')
right_join = weather.merge(station_info, left_on='station', right_on='id', how='right')
right_join[right_join.datatype.isna()].head()

,date,datatype,station,attributes,value,id,name,latitude,longitude,elevation
0,NaN,NaN,NaN,NaN,NaN,GHCND:US1CTFR0022,"STAMFORD 2.6 SSW, CT US",41.064100,-73.577000,36.6
344,NaN,NaN,NaN,NaN,NaN,GHCND:US1NJBG0001,"BERGENFIELD 0.3 SW, NJ US",40.921298,-74.001983,20.1
345,NaN,NaN,NaN,NaN,NaN,GHCND:US1NJBG0002,"SADDLE BROOK TWP 0.6 E, NJ US",40.902694,-74.083358,16.8
718,NaN,NaN,NaN,NaN,NaN,GHCND:US1NJBG0005,"WESTWOOD 0.8 ESE, NJ US",40.983041,-74.015858,15.8
719,NaN,NaN,NaN,NaN,NaN,GHCND:US1NJBG0006,"RAMSEY 0.6 E, NJ US",41.058611,-74.134068,112.2


In [15]:
left_join.sort_index(axis=1).sort_values(['date', 'station'], ignore_index=True).equals(right_join.sort_index(axis=1).sort_values(['date', 'station'], ignore_index=True))

True

In [16]:
get_row_count(inner_join, left_join, right_join)

[78780, 78949, 78949]

In [21]:
outer_join = weather.merge(station_info[station_info.id.str.contains('US1NY')], left_on='station', right_on='id', how='outer', indicator=True)
pd.concat([
    outer_join.query(f'_merge == "{kind}"').sample(2, random_state=42) for kind in outer_join._merge.unique()]).sort_index()

,date,datatype,station,attributes,value,id,name,latitude,longitude,elevation,_merge
11566,2018-03-13T00:00:00,SNOW,GHCND:US1NJMS0040,",,N,",69.0,NaN,NaN,NaN,NaN,NaN,left_only
24129,2018-03-29T00:00:00,PRCP,GHCND:US1NYNS0046,",,N,",0.0,GHCND:US1NYNS0046,"MASSAPEQUA PARK 1.2 N, NY US",40.698077,-73.449893,10.7,both
28646,2018-12-31T00:00:00,WESF,GHCND:US1NYWC0018,"T,,N,",0.0,GHCND:US1NYWC0018,"ARMONK 0.3 SE, NY US",41.129960,-73.708161,117.3,both
65396,2018-09-23T00:00:00,AWND,GHCND:USW00094745,",,W,",1.7,NaN,NaN,NaN,NaN,NaN,left_only
78780,NaN,NaN,NaN,NaN,NaN,GHCND:US1NYBX0025,"BRONX 1.5 NE, NY US",40.859856,-73.845694,14.9,right_only
78793,NaN,NaN,NaN,NaN,NaN,GHCND:US1NYRL0003,"NANUET 0.3 E, NY US",41.095511,-74.009897,86.3,right_only


In [23]:
dirty_data = pd.read_csv('dirty_data.csv', index_col='date').drop_duplicates().drop(columns='SNWD')
dirty_data.head()

,station,PRCP,SNOW,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01T00:00:00,?,0.0,0.0,5505.0,-40.0,NaN,NaN,NaN
2018-01-02T00:00:00,GHCND:USC00280907,0.0,0.0,-8.3,-16.1,-12.2,NaN,False
2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-4.4,-13.9,-13.3,NaN,False
2018-01-04T00:00:00,?,20.6,229.0,5505.0,-40.0,NaN,19.3,True
2018-01-05T00:00:00,?,0.3,NaN,5505.0,-40.0,NaN,NaN,NaN
